<a href="https://colab.research.google.com/github/FabioCabreraMendoza/FOOD101project/blob/main/FOOD101PROYECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q mlflow pyngrok tensorflow-datasets gradio

import os
import time
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import mlflow
from tensorflow.keras import layers, models, applications, optimizers, callbacks
from tensorflow.keras import mixed_precision
from pyngrok import ngrok

# --- CONFIGURACION GLOBAL ---
NGROK_AUTH_TOKEN = "35kIrE6CdClNMjARhepFUEm9bNT_2mEdYtARGGuQnNPVoLNDc"
IMG_SIZE = 224
BATCH_SIZE = 64  # Aumentado a 64 gracias a Mixed Precision
NUM_CLASSES = 101
LEARNING_RATE = 0.0001 # Tasa mas baja para ajuste fino
EPOCHS = 10      # Mas epocas para el dataset completo
RANDOM_SEED = 42
MLFLOW_PORT = 5000
ARTIFACT_PATH = "mlflow_artifacts"

# --- OPTIMIZACION DE HARDWARE ---
# Configurar politica de precision mixta (acelera entrenamiento y reduce uso de VRAM)
try:
    policy = mixed_precision.Policy('mixed_float16')
    mixed_precision.set_global_policy(policy)
    print("Politica de Precision Mixta activada: mixed_float16")
except Exception as e:
    print(f"Advertencia: No se pudo activar precision mixta. {e}")

# Establecer semillas
tf.random.set_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.8/76.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 753.9/753.9 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 11.0 MB/s eta 0:00:00
Politica de Precision Mixta activada: mixed_float16


In [2]:
def init_mlflow_infrastructure():
    """
    Inicializa el servidor MLflow y gestiona el tunel Ngrok.
    Mata procesos zombies para liberar el puerto 5000.
    """
    print("Configurando infraestructura de seguimiento...")

    # 1. Limpieza de procesos
    get_ipython().system_raw("pkill -f mlflow")
    ngrok.kill()

    # 2. Configuracion de red
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)

    if not os.path.exists(ARTIFACT_PATH):
        os.makedirs(ARTIFACT_PATH)

    # 3. Lanzar demonio MLflow
    # Redirigimos stderr a null para mantener limpia la celda
    get_ipython().system_raw(f"nohup mlflow ui --port {MLFLOW_PORT} > /dev/null 2>&1 &")
    time.sleep(5)

    # 4. Establecer tunel seguro
    try:
        public_url = ngrok.connect(MLFLOW_PORT, bind_tls=True, host_header="rewrite")
        print(f"Panel MLflow disponible en: {public_url}")
        mlflow.set_tracking_uri(f"http://127.0.0.1:{MLFLOW_PORT}")
    except Exception as e:
        print(f"Fallo critico al iniciar Ngrok: {e}")

init_mlflow_infrastructure()

Configurando infraestructura de seguimiento...
Panel MLflow disponible en: NgrokTunnel: "https://sabina-unhardenable-tellingly.ngrok-free.dev" -> "http://localhost:5000"


In [3]:
def preprocess_pipeline(image, label):
    """
    Funcion ETL mapeada a traves del dataset.
    Optimizado para rendimiento en GPU.
    """
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32) / 255.0
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label

def load_full_dataset():
    """
    Carga el dataset Food-101 completo (5GB).
    Utiliza estrategias de caching y prefetching para manejar grandes volumenes.
    """
    print("Iniciando descarga/carga de Food-101 (5GB). Esto puede tomar tiempo...")

    # Carga sin slicing para obtener el dataset completo
    (ds_train, ds_val), ds_info = tfds.load(
        'food101',
        split=['train', 'validation'],
        shuffle_files=True,
        as_supervised=True,
        with_info=True
    )

    autotune = tf.data.AUTOTUNE

    # Pipeline de Entrenamiento
    # 1. Map: Preprocesamiento paralelo
    # 2. Shuffle: Mezcla buffer de 1000 elementos
    # 3. Batch: Agrupa en lotes
    # 4. Prefetch: Prepara el siguiente lote en background
    ds_train = ds_train.map(preprocess_pipeline, num_parallel_calls=autotune)
    ds_train = ds_train.shuffle(1000).batch(BATCH_SIZE).prefetch(autotune)

    # Pipeline de Validacion (Sin shuffle)
    ds_val = ds_val.map(preprocess_pipeline, num_parallel_calls=autotune)
    ds_val = ds_val.batch(BATCH_SIZE).prefetch(autotune)

    return ds_train, ds_val, ds_info

# Cargar datos
train_ds, val_ds, dataset_info = load_full_dataset()
class_names = dataset_info.features['label'].names
print(f"Pipeline ETL listo. Total imagenes de entrenamiento: {dataset_info.splits['train'].num_examples}")

Iniciando descarga/carga de Food-101 (5GB). Esto puede tomar tiempo...


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/food101/incomplete.A9TBTK_2.0.0/food101-train.tfrecord*...:   0%|         …

Generating validation examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/food101/incomplete.A9TBTK_2.0.0/food101-validation.tfrecord*...:   0%|    …

Dataset food101 downloaded and prepared to /root/tensorflow_datasets/food101/2.0.0. Subsequent calls will reuse this data.
Pipeline ETL listo. Total imagenes de entrenamiento: 75750


In [4]:
def build_production_model():
    """
    Construye modelo MobileNetV2 adaptado para 101 clases.
    """
    base_model = applications.MobileNetV2(
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
        include_top=False,
        weights='imagenet'
    )
    # Importante: MobileNetV2 tiene capas BatchNormalization que deben mantenerse
    # en modo inferencia incluso si descongelamos pesos.
    base_model.trainable = False

    inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x) # Aumentado dropout para dataset grande

    # Capa de salida debe ser float32 explícitamente cuando se usa mixed_precision
    outputs = layers.Dense(NUM_CLASSES, activation='softmax', dtype='float32')(x)

    model = models.Model(inputs, outputs, name="Food101_Production_V1")

    model.compile(
        optimizer=optimizers.Adam(learning_rate=LEARNING_RATE),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

def get_callbacks():
    """
    Define estrategias de control de entrenamiento.
    """
    early_stop = callbacks.EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True,
        verbose=1
    )

    return [early_stop]

print("Arquitectura y callbacks definidos.")

Arquitectura y callbacks definidos.


In [5]:
def run_full_experiment(run_name="Full_Food101_Training"):

    mlflow.set_experiment("Food101_Production_Scale")

    with mlflow.start_run(run_name=run_name) as run:
        print(f"--- Iniciando Run: {run_name} ---")

        mlflow.tensorflow.autolog(log_models=True)

        model = build_production_model()
        my_callbacks = get_callbacks()

        # Entrenamiento
        # Nota: Al ser dataset completo, cada epoca tomara varios minutos
        history = model.fit(
            train_ds,
            validation_data=val_ds,
            epochs=EPOCHS,
            callbacks=my_callbacks,
            verbose=1
        )

        print(f"Run completado. ID: {run.info.run_id}")
        return model

# Ejecutar el entrenamiento principal
# Asegurate de tener la GPU activada en Colab (Runtime > Change runtime type > T4 GPU)
final_model = run_full_experiment(run_name="MobileNet_FullDataset_v1")

2025/11/30 02:03:17 INFO mlflow.tracking.fluent: Experiment with name 'Food101_Production_Scale' does not exist. Creating a new experiment.


--- Iniciando Run: MobileNet_FullDataset_v1 ---
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Epoch 1/10
1184/1184 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step - accuracy: 0.0906 - loss: 4.2709

1184/1184 ━━━━━━━━━━━━━━━━━━━━ 293s 218ms/step - accuracy: 0.0907 - loss: 4.2704 - val_accuracy: 0.4037 - val_loss: 2.6432
Epoch 2/10
1183/1184 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - accuracy: 0.3404 - loss: 2.7750

1184/1184 ━━━━━━━━━━━━━━━━━━━━ 240s 201ms/step - accuracy: 0.3405 - loss: 2.7749 - val_accuracy: 0.4887 - val_loss: 2.1457
Epoch 3/10
1184/1184 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step - accuracy: 0.4188 - loss: 2.3816

...

1184/1184 ━━━━━━━━━━━━━━━━━━━━ 188s 157ms/step - accuracy: 0.5240 - loss: 1.8899 - val_accuracy: 0.5820 - val_loss: 1.6367
Epoch 9/10
1184/1184 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - accuracy: 0.5305 - loss: 1.8601

1184/1184 ━━━━━━━━━━━━━━━━━━━━ 189s 157ms/step - accuracy: 0.5305 - loss: 1.8601 - val_accuracy: 0.5871 - val_loss: 1.6108
Epoch 10/10
1183/1184 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step - accuracy: 0.5362 - loss: 1.8262

1184/1184 ━━━━━━━━━━━━━━━━━━━━ 198s 165ms/step - accuracy: 0.5362 - loss: 1.8262 - val_accuracy: 0.5914 - val_loss: 1.5923
Restoring model weights from the end of the best epoch: 10.
1/1 ━━━━━━━━━━━━━━━━━━━━ 18s 18s/step


2025/11/30 02:42:26 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Run completado. ID: c4244dfd47164bb4bae9d201f5ffd729
🏃 View run MobileNet_FullDataset_v1 at: http://127.0.0.1:5000/#/experiments/202097377562280212/runs/c4244dfd47164bb4bae9d201f5ffd729
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/202097377562280212


In [6]:
import gradio as gr

def predict_service(image):
    if image is None:
        return "Error: No image provided"

    # Preprocesamiento
    img_array = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    img_array = tf.cast(img_array, tf.float32) / 255.0
    img_array = tf.expand_dims(img_array, axis=0)

    # Inferencia
    t_start = time.time()
    predictions = final_model.predict(img_array, verbose=0)
    inference_time = time.time() - t_start

    # Post-procesamiento
    score = tf.nn.softmax(predictions[0])
    top_3_indices = np.argsort(predictions[0])[::-1][:3]

    result = {}
    for i in top_3_indices:
        class_name = class_names[i]
        confidence = float(predictions[0][i])
        result[class_name] = confidence

    return result, f"{inference_time:.4f} seg"

# Definicion de UI
iface = gr.Interface(
    fn=predict_service,
    inputs=gr.Image(label="Imagen de Entrada"),
    outputs=[
        gr.Label(num_top_classes=3, label="Clasificacion"),
        gr.Number(label="Tiempo de Inferencia")
    ],
    title="Sistema de Reconocimiento Food-101",
    description="Modelo entrenado en 75,750 imagenes. Sube una foto de comida.",
    allow_flagging="never"
)

iface.launch(share=True, debug=False)


/usr/local/lib/python3.12/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://94aed32d3d1a065d0d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [7]:
# Liberacion de recursos
ngrok.kill()
get_ipython().system_raw("pkill -f mlflow")
print("Sesion finalizada y puertos liberados.")

Sesion finalizada y puertos liberados.


In [9]:
import os
import time
from pyngrok import ngrok

# 1. Matar cualquier proceso zombi anterior
print("🚑 Reviviendo el servidor MLflow...")
get_ipython().system_raw("pkill -f mlflow")
ngrok.kill()

# 2. Tu Token (Asegúrate de que sea el correcto)
NGROK_AUTH_TOKEN = "35kIrE6CdClNMjARhepFUEm9bNT_2mEdYtARGGuQnNPVoLNDc"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# 3. Lanzar MLflow de nuevo
# Usamos nohup para que sobreviva en background
get_ipython().system_raw("nohup mlflow ui --port 5000 > mlflow.log 2>&1 &")
time.sleep(5) # Esperar a que arranque

# 4. Reconectar Ngrok
try:
    public_url = ngrok.connect(5000, bind_tls=True, host_header="rewrite")
    print(f"✅ Servidor operativo de nuevo: {public_url}")
except Exception as e:
    print(f"❌ Error al reconectar: {e}")

# 5. Importante: Decirle a Python dónde está el servidor
import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")

🚑 Reviviendo el servidor MLflow...
✅ Servidor operativo de nuevo: NgrokTunnel: "https://sabina-unhardenable-tellingly.ngrok-free.dev" -> "http://localhost:5000"


In [10]:
# --- FASE 2: FINE-TUNING (AJUSTE FINO) ---

import tensorflow as tf
import mlflow

def run_finetuning(model_base, initial_epochs=10, new_epochs=10):

    # 1. Configurar MLflow para la nueva fase
    mlflow.set_experiment("Food101_Optimization")

    with mlflow.start_run(run_name="MobileNetV2_FineTuning_Top50") as run:
        print(f"--- Iniciando Fine-Tuning: {run.info.run_name} ---")

        # 2. Descongelar el modelo base
        # En nuestra arquitectura, la capa [1] es el MobileNetV2
        base_model = model_base.layers[1]
        base_model.trainable = True

        print(f"Capas totales en MobileNetV2: {len(base_model.layers)}")

        # Congelar todas las capas EXCEPTO las ultimas 50
        # Esto permite adaptar solo las caracteristicas de alto nivel (texturas de comida)
        fine_tune_at = len(base_model.layers) - 50
        for layer in base_model.layers[:fine_tune_at]:
            layer.trainable = False

        print(f"Descongelando desde la capa {fine_tune_at} hacia adelante.")

        # 3. Re-compilar con Learning Rate MUY BAJO (Crucial)
        # Usamos 1e-5 (0.00001) para no destruir los pesos pre-entrenados
        model_base.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

        # Activar log automatico
        mlflow.tensorflow.autolog(log_models=True)

        # 4. Continuar el entrenamiento
        # 'initial_epoch' asegura que las graficas de Loss sigan donde se quedaron
        total_epochs = initial_epochs + new_epochs

        history_fine = model_base.fit(
            train_ds,
            validation_data=val_ds,
            epochs=total_epochs,
            initial_epoch=initial_epochs,
            callbacks=get_callbacks(), # Usamos los mismos callbacks (EarlyStopping)
            verbose=1
        )

        print(f"Fine-Tuning completado. ID: {run.info.run_id}")
        return model_base, history_fine

# Ejecutar la mejora
# Asegurate de que 'final_model' esta cargado en memoria de la fase anterior
# Si se desconecto el entorno, tendras que volver a cargar/entrenar la fase 1 primero.
if 'final_model' in locals():
    improved_model, history_ft = run_finetuning(final_model, initial_epochs=10, new_epochs=10)
else:
    print("Error: No se encuentra 'final_model'. Por favor ejecuta la fase 1 (Entrenamiento Base) primero.")

2025/11/30 03:26:05 INFO mlflow.tracking.fluent: Experiment with name 'Food101_Optimization' does not exist. Creating a new experiment.


--- Iniciando Fine-Tuning: MobileNetV2_FineTuning_Top50 ---
Capas totales en MobileNetV2: 154
Descongelando desde la capa 104 hacia adelante.


Epoch 11/20
1184/1184 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step - accuracy: 0.4211 - loss: 2.4005

1184/1184 ━━━━━━━━━━━━━━━━━━━━ 325s 230ms/step - accuracy: 0.4211 - loss: 2.4003 - val_accuracy: 0.6024 - val_loss: 1.5134
Epoch 12/20
1183/1184 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step - accuracy: 0.5514 - loss: 1.7580

1184/1184 ━━━━━━━━━━━━━━━━━━━━ 209s 174ms/step - accuracy: 0.5515 - loss: 1.7580 - val_accuracy: 0.6294 - val_loss: 1.4009
Epoch 13/20
1183/1184 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - accuracy: 0.5926 - loss: 1.5858

...

1184/1184 ━━━━━━━━━━━━━━━━━━━━ 207s 173ms/step - accuracy: 0.6913 - loss: 1.1582 - val_accuracy: 0.6966 - val_loss: 1.1156
Epoch 19/20
1183/1184 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - accuracy: 0.7057 - loss: 1.1009

1184/1184 ━━━━━━━━━━━━━━━━━━━━ 204s 171ms/step - accuracy: 0.7057 - loss: 1.1009 - val_accuracy: 0.7021 - val_loss: 1.0943
Epoch 20/20
1183/1184 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - accuracy: 0.7194 - loss: 1.0466

1184/1184 ━━━━━━━━━━━━━━━━━━━━ 261s 169ms/step - accuracy: 0.7194 - loss: 1.0466 - val_accuracy: 0.7074 - val_loss: 1.0767
Restoring model weights from the end of the best epoch: 20.
1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step


2025/11/30 04:07:23 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Fine-Tuning completado. ID: a358632e28a04da987219026574ff20e
🏃 View run MobileNetV2_FineTuning_Top50 at: http://127.0.0.1:5000/#/experiments/178703244249790697/runs/a358632e28a04da987219026574ff20e
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/178703244249790697


In [11]:
# --- DESPLIEGUE FINAL (GRADIO) ---
import gradio as gr
import tensorflow as tf
import numpy as np

# 1. Asegurarnos de usar el mejor modelo disponible en memoria
# Tu función devolvió el modelo entrenado, vamos a usar ese.
if 'improved_model' in locals():
    model_para_demo = improved_model
    print("✅ CARGADO: Modelo Optimizado (Fine-Tuning ~70% acc)")
elif 'final_model' in locals():
    model_para_demo = final_model
    print("⚠️ AVISO: Usando modelo base (Fase 1). Si hiciste Fine-Tuning, asegúrate de haber asignado el resultado a una variable.")
else:
    print("❌ ERROR: No hay modelo en memoria. Ejecuta el entrenamiento primero.")

# 2. Función de Predicción (La lógica del cerebro)
def predecir_plato(imagen):
    if imagen is None:
        return None

    # Preprocesar igual que en el entrenamiento
    img = tf.image.resize(imagen, (224, 224))
    img = tf.cast(img, tf.float32) / 255.0
    img = tf.expand_dims(img, axis=0)

    # Predecir
    predicciones = model_para_demo.predict(img)

    # Obtener las 3 clases más probables
    top_3_indices = np.argsort(predicciones[0])[::-1][:3]

    resultado = {}
    for i in top_3_indices:
        nombre_clase = class_names[i] # Usamos la lista de nombres que cargamos al inicio
        confianza = float(predicciones[0][i])
        resultado[nombre_clase] = confianza

    return resultado

# 3. Interfaz Visual
demo = gr.Interface(
    fn=predecir_plato,
    inputs=gr.Image(type="numpy", label="Sube una foto de comida"),
    outputs=gr.Label(num_top_classes=3, label="Top 3 Predicciones"),
    title="🍔 Food-101 Classifier (MobileNetV2 Fine-Tuned)",
    description="Modelo de IA optimizado capaz de reconocer 101 tipos de comida con un 70% de precisión.",
    theme="soft"
)

# 4. Lanzar (Genera link público)
print("Generando enlace público...")
demo.launch(share=True, debug=False)

✅ CARGADO: Modelo Optimizado (Fine-Tuning ~70% acc)
Generando enlace público...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c71fe7b8def6227be4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
